In [3]:
import struct
import sys
from pathlib import Path
import numpy as np
import open3d as o3d
import copy

In [4]:
bath = (
    Path()
    .cwd()
    .parents[0]
    .joinpath("3rdparty/Open3D/examples/test_data/bathtub_0154.ply")
)

In [3]:
with open(bath, "rb") as f:
    # read header
    while True:
        line = f.readline()
        print(line)
        if b"end_header" in line:
            break
        if b"vertex " in line:
            vnum = int(line.split(b" ")[-1])  # num of vertices
        if b"face " in line:
            fnum = int(line.split(b" ")[-1])  # num of faces

    # read vertices
    for i in range(vnum):
        for j in range(3):
            print(struct.unpack("f", f.read(4))[0], end=" ")
        print("")

    # read faces
    for i in range(fnum):
        n = struct.unpack("B", f.read(1))[0]
        for j in range(n):
            print(struct.unpack("i", f.read(4))[0], end=" ")
        print("")

b'ply\n'
b'format binary_little_endian 1.0\n'
b'comment VCGLIB generated\n'
b'element vertex 1494\n'
b'property float x\n'
b'property float y\n'
b'property float z\n'
b'element face 1194\n'
b'property list uchar int vertex_indices\n'
b'end_header\n'
-9208.080078125 19354.849609375 -6295.0498046875 
-1696.699951171875 -3551.25 -6295.0498046875 
9129.2998046875 19354.849609375 -6295.0498046875 
-1873.699951171875 -3978.550048828125 -6295.0498046875 
-9208.080078125 -28457.44921875 -6295.0498046875 
-1415.199951171875 -3184.35009765625 -6295.0498046875 
-1048.300048828125 -2902.85009765625 -6295.0498046875 
-621.0 -2725.85009765625 -6295.0498046875 
-162.39999389648438 -2665.449951171875 -6295.0498046875 
-1934.0999755859375 -4437.14990234375 -6295.0498046875 
-1873.699951171875 -4895.64990234375 -6295.0498046875 
-1696.699951171875 -5322.9501953125 -6295.0498046875 
-1415.199951171875 -5689.85009765625 -6295.0498046875 
-1048.300048828125 -5971.4501953125 -6295.0498046875 
-621.0 -6148.3

In [6]:
str(bath)

'/Users/argon/dev/books/3dpcp_book_codes/3rdparty/Open3D/examples/test_data/bathtub_0154.ply'

In [ ]:
pcd = o3d.io.read_point_cloud(str(bath))
print(pcd)

o3d.visualization.draw_geometries([pcd])

PointCloud with 1494 points.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


６行目で軸を表す矢印のメッシュデータを作っています．この部分を適当な点群データに置き換えてもかまいません．赤い矢印が x 軸，緑の矢印が y 軸，青い矢印が z 軸を表しており，すべての矢印の長さは1.0になっています．次に，９～16行目ではこのメッシュデータを回転させています．Open3Dでは，回転行列 R を用いて回転を指定します．R の値は直接入力してもよいですが，他の回転表現を R に変換する関数も用意されています．９行目は yxz 系のオイラー角を入力として R を出力しています．この例では，y 軸まわりに角度π/3 だけ回転し，他の軸では回転していません．11行目は回転軸に対して角度（スカラ）を乗算したベクトルを入力としています．この例でも回転軸が y 軸，角度がπ/3 です．13行目は四元数を入力しています．この例も y 軸まわりに角度π/3 だけ回転する場合の四元数となっています．すなわち，これらの行はすべて同じ R の値を出力します．最後に，16行目で回転行列 R を入力してデータを回転させます．この関数の第２引数 center は回転中心を表しており，この例では原点 [0, 0, 0] を指定しています．もし第２引数を与えない場合は，デフォルトの挙動として，点群データの重心を中心とした回転が適用されます． 　次に，20行目では回転後のメッシュデータをベクトル [0.5, 0.7, 1] だけ並進させています．回転と並進を同時に行うことも可能です．25～28行目は同次変換行列を作成して元のメッシュデータに適用しています．得られる結果は20行目の操作とまったく同じものになります．最後に，34行目でメッシュデータを0.5倍にスケール変換しています．この例ではメッシュデータの中心を変換の中心としていますが，他の点，例えば原点を基準にスケール変換することも可能です．その場合は，原点からメッシュデータまでの距離も0.5倍に縮小されるでしょう．

金崎朝子; 秋月秀一; 千葉直也. 詳解　３次元点群処理　Ｐｙｔｈｏｎによる基礎アルゴリズムの実装 (ＫＳ理工学専門書) (p.52). 講談社. Kindle 版. 

In [7]:
mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()

In [8]:
# Rotate
R = o3d.geometry.get_rotation_matrix_from_yxz([np.pi / 3, 0, 0])
print("R:", np.round(R, 7))
R = o3d.geometry.get_rotation_matrix_from_axis_angle([0, np.pi / 3, 0])
print("R:", np.round(R, 7))
R = o3d.geometry.get_rotation_matrix_from_quaternion(
    [np.cos(np.pi / 6), 0, np.sin(np.pi / 6), 0]
)
print("R:", np.round(R, 7))
mesh_r = copy.deepcopy(mesh)
mesh_r.rotate(R, center=[0, 0, 0])

R: [[ 0.5        0.         0.8660254]
 [ 0.         1.         0.       ]
 [-0.8660254  0.         0.5      ]]
R: [[ 0.5        0.         0.8660254]
 [ 0.         1.         0.       ]
 [-0.8660254  0.         0.5      ]]
R: [[ 0.5        0.         0.8660254]
 [ 0.         1.         0.       ]
 [-0.8660254  0.         0.5      ]]


TriangleMesh with 1134 points and 2240 triangles.

In [13]:
# Translate
t = [0.5, 0.7, 1]
mesh_t = copy.deepcopy(mesh_r).translate(t)
print("Type q to continue.")
o3d.visualization.draw_geometries([mesh, mesh_t])


Type q to continue.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [12]:
# Rotate and translate
T = np.eye(4)
T[:3, :3] = R
T[:3, 3] = t
mesh_t = copy.deepcopy(mesh).transform(T)
print("Type q to continue.")
o3d.visualization.draw_geometries([mesh, mesh_t])

Type q to continue.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [ ]:
# Scale
mesh_s = copy.deepcopy(mesh_t)
mesh_s.scale(0.5, center=mesh_s.get_center())
print("Type q to exit.")
o3d.visualization.draw_geometries([mesh, mesh_s])

Type q to exit.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


### 三次元データの点の扱いの難しさについて
本節では、最も基礎的な点群処理の1つであるサンプリングについて学びましょう。2次元画像と3次元点群データの最も大きな違いはデータ構造にあるといっても過言ではありません。2次元画像は、画像を構成する点（ピクセル）が格子状に並んでおり、そのすべての点が輝度やRGBカラー値といった値を持っています。2次元画像のピクセルの数は画像の大きさと直接の関係があります。例えば縦横のアスペクト比を変えずに画像の解像度を2倍にすれば、ピクセルの数は4倍になります。一方で、3次元点群データは任意間隔の計測点の3次元座標データの集合です。何らかの3次元計測によってデータを取得した場合、計測器に近い物体の表面上の点と点の距離は小さく、遠物体の表面上の点と点の距離は大きくなります。また、現実世界の計測で得られた3次元点群の場合には、手前の物体に遮蔽された物体の表面上の点は計測できないため、欠きな状態になります。このように、3次元点群データは空間的に不均一に存在しており、また、点の数が多いからといって解像度が高いとも限りません。点群データを整列したデータとして扱う1つの方法は、等間隔サンプリングによるボクセル化です。ボクセルとは、ちょうど2次元画像のピクセルの概念を3次元に拡張したようなものになっており、xyz軸方向それぞれに点が整列しています。ただし、2次元画像のピクセルが常に値を持っているのに対し、ボクセルデータは、計測点の存在しない箇所のボクセルが空である（値を持たない）ため、疎なデータであることには注意が必要です。このため、たとえ点群データをボクセル化したとしても、データ全体に対して何らかの均等な処理を加えたりデータの性質を解析したりするような操作は、2次元画像ほど容易ではありません。

In [20]:
pcd = o3d.io.read_point_cloud(str(bath))

In [ ]:
o3d.visualization.draw_geometries(
    [pcd],
    zoom=0.3412,
    front=[0.4257, -0.2125, -0.8795],
    lookat=[2.6172, 2.0475, 1.532],
    up=[-0.0694, -0.9768, 0.2024],
)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


PointCloud with 400 points.


In [35]:
downpcd = pcd.voxel_down_sample(voxel_size=1e3)
print(downpcd)

o3d.visualization.draw_geometries(
    [downpcd],
    zoom=0.3412,
    front=[0.4257, -0.2125, -0.8795],
    lookat=[2.6172, 2.0475, 1.532],
    up=[-0.0694, -0.9768, 0.2024],
)

PointCloud with 209 points.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


この例では、まず fragment.ply ファイルの3次元点群データを読み込んで表示します。次に、表示ウィンドウ上でqキーを押すなどしてウィンドウを閉じると、1辺の大きさ 0.03 のボクセルデータとして等間隔にサンプリングした点群を作成し、表示します。点の総数が減り、まばらになっているのが見てわかります。ボクセル化の処理手順は下記のとおりです。まず、点群データ内のすべての点 xyz 座標の最大最小値を取得します。これらの値のすべての組み合わせからなる8点を頂点とする直方体をバウンディングボックスと呼びます。そして、1辺の大きさ 0.03 のボクセルでバウンディングボックスを分割します。次に、各点の座標を参照してボクセルに割り当てられるインデックスを計算し、そのボクセルへと点を追加していきます。最後に、各ボクセル内の点群の座標の平均値を求め、ボクセルごとに0ないし1個の新しい点を作成して点群データを出力します。
なお、Open3Dには voxel_down_sample() の他に uniform_down_sample() という関数が存在しますが、これは本書で使用している用語「等間隔サンプリング」とは別物であることに注意が必要です。本書では、空間的に等間隔なサンプリングを行うという意味で「等間隔サンプリング」という用語を使用しています。これはつまり、処理の内容としてはボクセル化と相違ありません注7。これに対して、Open3Dの関数 uniform_down_sample() は入力点群のデータの並びに沿って等間隔にデータをサンプリングします。引数としてインデックスのステップ数 k をとり、例えば k = 3 としたときは、入力点群の点を2個飛ばしでサンプリングして全体の3分の1の個数の点群データを出力します。点群データの点の並びは一般的には規則性がありません。このため、入力点群データの点が空間的に等間隔に並んでいない限り、出力点群データも空間的に等間隔にはなりません。

#### Farthest Point Sampling
この手法の概要は以下の通り。
1. まず、点群データの中からランダムに1点を選び、その点を出発点とする。
2. 出発点から最も遠い点を選び、その点を次の出発点とする。
3. 2の操作を繰り返し、指定した点の数だけ点を選択する。
4. 選択された点を出力する。


In [3]:
def l2_norm(a, b):
    return ((a - b) ** 2).sum(axis=1)


def farthest_point_sampling(pcd, k, metrics=l2_norm):
    indices = np.zeros(k, dtype=np.int32)
    points = np.asarray(pcd.points)
    distances = np.zeros((k, points.shape[0]), dtype=np.float32)
    indices[0] = np.random.randint(len(points))
    farthest_point = points[indices[0]]
    min_distances = metrics(farthest_point, points)
    distances[0, :] = min_distances
    for i in range(1, k):
        indices[i] = np.argmax(min_distances)
        farthest_point = points[indices[i]]
        distances[i, :] = metrics(farthest_point, points)
        min_distances = np.minimum(min_distances, distances[i, :])
    pcd = pcd.select_by_index(indices)
    return pcd


In [5]:
# main
filename = str(bath)
# 点の数
k = 100
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

# o3d.visualization.draw_geometries(
#     [pcd],
#     zoom=0.3412,
#     front=[0.4257, -0.2125, -0.8795],
#     lookat=[2.6172, 2.0475, 1.532],
#     up=[-0.0694, -0.9768, 0.2024],
# )

downpcd = farthest_point_sampling(pcd, k)
print(downpcd)

o3d.visualization.draw_geometries(
    [downpcd],
    zoom=0.3412,
    front=[0.4257, -0.2125, -0.8795],
    lookat=[2.6172, 2.0475, 1.532],
    up=[-0.0694, -0.9768, 0.2024],
)

Loading a point cloud from /Users/argon/dev/books/3dpcp_book_codes/3rdparty/Open3D/examples/test_data/bathtub_0154.ply
PointCloud with 1494 points.


NameError: name 'farthest_point_sampling' is not defined

### Poisson Disk Sampling (PDS)
2.4.2 節では出力点群の点の個数を指定して均一なサンプリングを行う FPS という手法を紹介しましたが、FPS は計算コストが高いという点が難点です。このため、学習時の前処理などのオフライン処理に適しています。しかし、高速な処理速度が必要とされるオンライン処理におけるサンプリングでは、別の方法を考えねばなりません。1つの単純な解決としてはランダムサンプリングが挙げられますが、空間的な均一性は保証されません。そこで、本節では Poisson Disk Sampling (PDS) というサンプリング手法を紹介します。この方法はランダムサンプリングに近いもので、サンプリングされた点の2点間距離の最小値を、ある指定した値以下にならないように制御できます。このため、出力点群のすべての点が互いにある一定以上の距離で離れており、実用的には、空間的均一性の高いサンプリングを行うことができます。
PDS では、はじめに1点をランダムに選択します。そして、その次の点もランダムに1点を選択します。このとき、選択された点を中心とする半径 r の球に注目し、球の中にすでにサンプリングされた点があれば、選択された点を削除します。この操作を、選択された点の数が k 個に達するまで繰り返します。

In [6]:
print("Loading a triangle mesh from", filename)
mesh = o3d.io.read_triangle_mesh(filename)
print(mesh)

o3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True)

downpcd = mesh.sample_points_poisson_disk(number_of_points=k)
print(downpcd)

o3d.visualization.draw_geometries([downpcd])

Loading a triangle mesh from /Users/argon/dev/books/3dpcp_book_codes/3rdparty/Open3D/examples/test_data/bathtub_0154.ply
TriangleMesh with 1494 points and 1194 triangles.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
PointCloud with 100 points.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


2024-11-23 21:58:37.629 python[30447:459537] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-23 21:58:37.629 python[30447:459537] +[IMKInputSession subclass]: chose IMKInputSession_Modern


### 外れ値処理は割愛

### 法線推定

点群の法線ベクトルを推定しましょう。さまざまな点群処理において、法線ベクトルはとても重要な情報となります。例えば、物体認識を行うためには特徴量を抽出する必要がありますが、点を表す特徴量として、（RGBカラー値とともに）法線の値がよく利用されます。あるいは、点群の位置合わせを行う際に面と面を合わせる処理が入ることがありますが、この際に法線の情報が必要になります。法線/法線ベクトルとは、従来、直線や平面に対して定義されるものであり、点に対して定義できるものではありません。点群の法線を求めるときは、暗にその点群が何らかの物体表面という曲面上に分布していることを仮定します。そして、各点の法線は、その点における接平面に直交するベクトルとして定義されます。
点群の法線を求める方法を説明しましょう。まず、各点の近傍点を求めます。そして、近傍点群の3次元座標に対して主成分分析を行います。主成分分析は、近傍点群の分散共分散を求め、その固有値分解を行うという分析手法です。通常の場合、例えば次元削減を行うために主成分分析を行う場合は、固有値の大きいものから順に任意個の固有ベクトルを抽出します。固有値の大きい固有ベクトルは、すなわちサンプルの分散が大きい軸を表します。今回の場合では、法線ベクトルとして、近傍点群の分散が最も小さい軸を選びます。すなわち、最小固有値を持つ固有ベクトルを選べば、これが求める法線ベクトルになります。

In [ ]:
knot = Path().cwd().parents[0].joinpath("3rdparty/Open3D/examples/test_data/knot.ply")

In [9]:
filename = str(knot)

mesh = o3d.io.read_triangle_mesh(filename)
print(mesh)
o3d.visualization.draw_geometries([mesh])

pcd = o3d.geometry.PointCloud()
pcd.points = mesh.vertices
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=10.0, max_nn=10)
)

print(np.asarray(pcd.normals))
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

mesh.compute_vertex_normals()

print(np.asarray(mesh.triangle_normals))
o3d.visualization.draw_geometries([mesh])


TriangleMesh with 1440 points and 2880 triangles.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[[ 0.61135989 -0.77171743 -0.17518931]
 [ 0.86306727 -0.42647589  0.27061633]
 [ 0.90355467 -0.32161852  0.28310863]
 ...
 [ 0.49058774  0.39790026  0.77524128]
 [ 0.07380545  0.69726419  0.71300449]
 [ 0.3566114  -0.81529877 -0.45619756]]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[[ 0.79164373 -0.53951444  0.28674793]
 [ 0.8319824  -0.53303008  0.15389681]
 [ 0.83488162 -0.09250101  0.54260136]
 ...
 [ 0.16269924 -0.76215917 -0.6266118 ]
 [ 0.52755226 -0.83707495 -0.14489352]
 [ 0.56778973 -0.76467734 -0.30476777]]
[Open3D WARNIN